<center> <b> Introduction to Data Management with Python </b> </center>
<center> Assignment 03 </center>
<center> Floriann Louis </center>
<center> <i> 05-Dec-2022 </i> </center>

**Homework: SQLite database**

• Using the same dataset you used for Assignment-2 (same groups), and the table of “country-continents” (see link [1]), you are going to create a SQLite database to answer the questions in the next slide.

• The “countries” table contain the list of countries and the continent they belong.

• Choose one measurement of your (OWID) dataset for this exercise.

In [67]:
# Import required libraries

import sqlite3
import requests
import io
import pandas as pd

In [68]:
# Load vaccination data

data_vcn = pd.read_csv("data/data_vcn.csv")
data_vcn = data_vcn.loc[:, ~data_vcn.columns.str.contains('^Unnamed')]
data_vcn.head(2)

iso_code     location        date  total_vaccinations
0      AFG  Afghanistan  2021-02-22                   0
1      AFG  Afghanistan  2021-02-23                   0

In [69]:
# Load continent data

data_cnt = pd.read_csv("data/data_cnt.csv")
data_cnt = data_cnt.loc[:, ~data_cnt.columns.str.contains('^Unnamed')]
data_cnt.head(2)

location iso_code continent    sub_continent
0    Afghanistan      AFG      Asia    Southern Asia
1  Åland Islands      ALA    Europe  Northern Europe

### Insert vaccination and continent tables/dataset to 'data' database

In [71]:
#Establish connection to the db

con = sqlite3.connect('data.db')

In [72]:
# Create the table
cursor = con.cursor()

# Create table vaccination
cursor.execute('''CREATE TABLE vaccination (
                                iso_code TEXT,
                                location TEXT,
                                date datetime,
                                total_vaccinations NUMERIC
                                )'''
                )

# Create table continent
cursor.execute('''CREATE TABLE continent (
                                location TEXT,
                                iso_code TEXT,
                                continent TEXT,
                                sub_continent TEXT
                                )'''
                )

con.commit()

In [73]:
# Insert data into table

lst_vaccination = data_vcn.values.tolist()
lst_continent = data_cnt.values.tolist()

try:
    cursor.executemany("INSERT INTO vaccination VALUES (?,?,?,?)", lst_vaccination)
    cursor.executemany("INSERT INTO continent VALUES (?,?,?,?)", lst_continent)
except Exception as E:
    print('Error : ', E)
else:
    con.commit()
    print('data inserted')

data inserted


### Select from database

In [76]:
#Checking all the values in vaccination table

cursor.execute('''SELECT * FROM vaccination
                LIMIT 2
                ''')
result = cursor.fetchall()
result

[('AFG', 'Afghanistan', '2021-02-22', 0),
 ('AFG', 'Afghanistan', '2021-02-23', 0)]

In [77]:
#Checking all the values in continent table

cursor.execute('''SELECT * FROM continent
                LIMIT 2
                ''')
result = cursor.fetchall()
result

[('Afghanistan', 'AFG', 'Asia', 'Southern Asia'),
 ('Åland Islands', 'ALA', 'Europe', 'Northern Europe')]

### Select query by joining vaccination and continent dataset

In [166]:
#Joining both the tables
cursor.execute("""
            SELECT *
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            LIMIT 2
            """)
covid_full_retrieved = cursor.fetchall()
covid_full_retrieved

[('AFG',
  'Afghanistan',
  '2021-02-22',
  0,
  'Afghanistan',
  'AFG',
  'Asia',
  'Southern Asia'),
 ('AFG',
  'Afghanistan',
  '2021-02-23',
  0,
  'Afghanistan',
  'AFG',
  'Asia',
  'Southern Asia')]

### 1. How many countries per continent? Order by continent name.

In [163]:
cursor.execute("""
            SELECT continent, continent.location
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            GROUP BY vaccination.iso_code
            ORDER BY continent.location
            """)
covid_full_retrieved = cursor.fetchall()
print(len(covid_full_retrieved))
covid_full_retrieved

217


[('Asia', 'Afghanistan'),
 ('Europe', 'Albania'),
 ('Africa', 'Algeria'),
 ('Europe', 'Andorra'),
 ('Africa', 'Angola'),
 ('Americas', 'Anguilla'),
 ('Americas', 'Antigua and Barbuda'),
 ('Americas', 'Argentina'),
 ('Asia', 'Armenia'),
 ('Americas', 'Aruba'),
 ('Oceania', 'Australia'),
 ('Europe', 'Austria'),
 ('Asia', 'Azerbaijan'),
 ('Americas', 'Bahamas'),
 ('Asia', 'Bahrain'),
 ('Asia', 'Bangladesh'),
 ('Americas', 'Barbados'),
 ('Europe', 'Belarus'),
 ('Europe', 'Belgium'),
 ('Americas', 'Belize'),
 ('Africa', 'Benin'),
 ('Americas', 'Bermuda'),
 ('Asia', 'Bhutan'),
 ('Americas', 'Bolivia'),
 ('Americas', 'Bonaire, Sint Eustatius and Saba'),
 ('Europe', 'Bosnia And Herzegovina'),
 ('Africa', 'Botswana'),
 ('Americas', 'Brazil'),
 ('Asia', 'Brunei Darussalam'),
 ('Europe', 'Bulgaria'),
 ('Africa', 'Burkina Faso'),
 ('Africa', 'Burundi'),
 ('Africa', 'Cabo Verde'),
 ('Asia', 'Cambodia'),
 ('Africa', 'Cameroon'),
 ('Americas', 'Canada'),
 ('Americas', 'Cayman Islands'),
 ('Africa', '

In [174]:
# select continent, count(location) from continent group by continent
cursor.execute("""
            SELECT continent, count ( DISTINCT vaccination.location)
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            GROUP BY continent
            ORDER BY continent
            """)
covid_full_retrieved = cursor.fetchall()
print(len(covid_full_retrieved))
covid_full_retrieved

5


[('Africa', 54),
 ('Americas', 47),
 ('Asia', 50),
 ('Europe', 48),
 ('Oceania', 18)]

In [165]:
# select continent, count(location) from continent group by continent
cursor.execute("""
            SELECT continent, continent.location, count (DISTINCT vaccination.location)
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            GROUP BY vaccination.iso_code
            ORDER BY continent.continent
            """)
covid_full_retrieved = cursor.fetchall()
print(len(covid_full_retrieved))
covid_full_retrieved

217


[('Africa', 'Angola', 1),
 ('Africa', 'Burundi', 1),
 ('Africa', 'Benin', 1),
 ('Africa', 'Burkina Faso', 1),
 ('Africa', 'Botswana', 1),
 ('Africa', 'Central African Republic', 1),
 ('Africa', "Côte D'Ivoire", 1),
 ('Africa', 'Cameroon', 1),
 ('Africa', 'Congo (Democratic Republic Of The)', 1),
 ('Africa', 'Congo', 1),
 ('Africa', 'Comoros', 1),
 ('Africa', 'Cabo Verde', 1),
 ('Africa', 'Djibouti', 1),
 ('Africa', 'Algeria', 1),
 ('Africa', 'Egypt', 1),
 ('Africa', 'Ethiopia', 1),
 ('Africa', 'Gabon', 1),
 ('Africa', 'Ghana', 1),
 ('Africa', 'Guinea', 1),
 ('Africa', 'Gambia', 1),
 ('Africa', 'Guinea Bissau', 1),
 ('Africa', 'Equatorial Guinea', 1),
 ('Africa', 'Kenya', 1),
 ('Africa', 'Liberia', 1),
 ('Africa', 'Libya', 1),
 ('Africa', 'Lesotho', 1),
 ('Africa', 'Morocco', 1),
 ('Africa', 'Madagascar', 1),
 ('Africa', 'Mali', 1),
 ('Africa', 'Mozambique', 1),
 ('Africa', 'Mauritania', 1),
 ('Africa', 'Mauritius', 1),
 ('Africa', 'Malawi', 1),
 ('Africa', 'Namibia', 1),
 ('Africa', 'N

### 2. Which countries have null (“quantity”) data entries, and how many?

In [162]:
# select location, count(total_vaccinations), total_vaccinations from vaccination where total_vaccinations = 0 group by location
cursor.execute("""
            SELECT vaccination.location, total_vaccinations, count(total_vaccinations)
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            WHERE vaccination.total_vaccinations = 0
            GROUP BY vaccination.location
            LIMIT 10
            """)
covid_full_retrieved = cursor.fetchall()
covid_full_retrieved

[('Afghanistan', 0, 544),
 ('Albania', 0, 382),
 ('Algeria', 0, 557),
 ('Andorra', 0, 574),
 ('Angola', 0, 528),
 ('Anguilla', 0, 564),
 ('Antigua and Barbuda', 0, 484),
 ('Armenia', 0, 370),
 ('Aruba', 0, 165),
 ('Australia', 0, 37)]

### 3. What is the “quantity” average and standard-deviation per country?

In [160]:
cursor.execute("""
            SELECT vaccination.location, round(AVG(total_vaccinations))
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            GROUP BY vaccination.location
            LIMIT 10
            """)
covid_full_retrieved = cursor.fetchall()
covid_full_retrieved

[('Afghanistan', 700440.0),
 ('Albania', 554843.0),
 ('Algeria', 510954.0),
 ('Andorra', 7292.0),
 ('Angola', 1266560.0),
 ('Anguilla', 1790.0),
 ('Antigua and Barbuda', 12946.0),
 ('Argentina', 62939500.0),
 ('Armenia', 109284.0),
 ('Aruba', 109933.0)]

### 4. What are the overall “quantity” average per continent? Order by average (descending).

In [159]:
cursor.execute("""
            SELECT continent, round(AVG(total_vaccinations)) as AVG
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            GROUP BY continent.continent
            ORDER BY AVG DESC
            """)
covid_full_retrieved = cursor.fetchall()
covid_full_retrieved

[('Asia', 95970393.0),
 ('Americas', 25131245.0),
 ('Europe', 15193654.0),
 ('Oceania', 2280111.0),
 ('Africa', 1348200.0)]

### 5. Which countries have the maximum/minimum of “quantity” in all history per continent (i.e., the top-1 country per continent of all times)? Order by continent name (ascending).

In [172]:
cursor.execute("""
            SELECT continent, vaccination.location, MAX(total_vaccinations) as MAX
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            GROUP BY continent.continent
            ORDER BY continent
            """)
covid_full_retrieved = cursor.fetchall()
covid_full_retrieved

[('Africa', 'Egypt', 100448725),
 ('Americas', 'United States', 646524294),
 ('Asia', 'China', 3441334000),
 ('Europe', 'Germany', 188878518),
 ('Oceania', 'Australia', 63461493)]

In [173]:
cursor.execute("""
            SELECT continent, vaccination.location, MIN(total_vaccinations) as MIN
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            GROUP BY continent.continent
            ORDER BY continent
            """)
covid_full_retrieved = cursor.fetchall()
covid_full_retrieved

[('Africa', 'Algeria', 0),
 ('Americas', 'Anguilla', 0),
 ('Asia', 'Afghanistan', 0),
 ('Europe', 'Albania', 0),
 ('Oceania', 'Australia', 0)]

### 6. Which countries have “quantity” greater/lesser than the continental average in the last year (or month) of measurement? Order by continent name (ASC) and by “quantity” descending/ascending.

In [189]:
cursor.execute("""
            SELECT vaccination.date, continent, vaccination.location, total_vaccinations as qty
            FROM vaccination
            LEFT JOIN continent on continent.iso_code = vaccination.iso_code
            WHERE strftime('%Y', vaccination.date) = strftime('%Y', date()) 
            GROUP BY vaccination.location, continent
            
            HAVING qty > (select avg(v2.total_vaccinations) 
            FROM vaccination v2 Left join continent c2 on v2.location = c2.location  
            where strftime('%Y', v2.date) = strftime('%Y', date()) AND continent.continent =c2.continent 
            GROUP BY c2.continent)
            
            ORDER BY continent ASC, qty DESC
            LIMIT 5
            """)
covid_full_retrieved = cursor.fetchall()
covid_full_retrieved

[('2022-01-01', 'Africa', 'South Africa', 27966664),
 ('2022-01-01', 'Africa', 'Tunisia', 12182712),
 ('2022-01-01', 'Africa', 'Ethiopia', 10916348),
 ('2022-01-01', 'Africa', 'Kenya', 10121263),
 ('2022-01-01', 'Africa', 'Zimbabwe', 7276239)]